In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
from collections import Counter
import warnings
from pandasgui import show
import gc
import catboost as cb
import os
import ipywidgets
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [ ]:
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
        return files

In [ ]:
file_names = (file_name('per_day'))

In [ ]:
opop = 1
for name in (file_names):
    print(name)
    if name == "175642r.txt": 
        opop = 0
    if (opop == 1): 
        continue
    data = pd.read_csv('per_day/'+name,header=None)
    if (data.loc[:,1].nunique()==1):
        continue
    X = data.loc[:,2:83]
    y = data.loc[:,1]
    op = 1
    i = 2
    while (op == 1):
        X_train, X_ver, y_train, y_ver = train_test_split(X, y, test_size=0.2, random_state=i)
        if y_ver.nunique() <= y_train.nunique():
            op = 0
        i += 2
        
    #categorical_features_indices = np.where(X_train.dtypes != np.float)[0]
    w = np.array((y_train+1))
    model = cb.CatBoostClassifier(iterations=500, depth=5, learning_rate=0.8, loss_function='MultiClass',logging_level='Verbose')
    model.fit(X_train,y_train,eval_set=(X_ver,y_ver),sample_weight=w,early_stopping_rounds=30,verbose=False)
    save_name = name.replace('r','')
    model.save_model('per_day_model/mo'+save_name)